# **Pydantic**
Pydantic, Python'da veri doğrulama ve ayar yönetimi için kullanılan bir kütüphanedir. Kütüphane, Python veri modelleri oluşturmayı kolaylaştırır ve
bu modeller aracılığıyla verilerin doğruluğunu, tutarlılığını ve tip güvenliğini sağlamaya yardımcı olur.

## Temel Özellikler
### Veri Doğrulama ve Dönüştürme:

*   Pydantic, verilen veri türlerini otomatik olarak doğrular ve gerekli dönüştürmeleri yapar. Örneğin, bir str tipi bir değeri int olarak belirtirseniz, Pydantic bu değeri otomatik olarak dönüştürür.

### Veri Modelleri:
*   Pydantic, veri modelleri oluşturmak için BaseModel sınıfını kullanır. Bu modeller, Python dataclass yapılarına benzer, ancak daha gelişmiş doğrulama ve dönüştürme yeteneklerine sahiptir.

### JSON ve Diğer Veri Formatlarıyla Çalışma:
*   Pydantic, JSON ve diğer veri formatlarını kolayca işleyebilir. Modellerinizi JSON, dict ve diğer formatlar arasında kolayca dönüştürebilirsiniz.

### Ayar Yönetimi:
*   Pydantic, uygulama ayarlarını yönetmek için de kullanılabilir. Örneğin, çevresel değişkenlerden veya konfigürasyon dosyalarından ayarları yükleyebilirsiniz.

# Examples

## Example 1

In [1]:
from datetime import datetime
from typing import Tuple

from pydantic import BaseModel


class Delivery(BaseModel):
    timestamp: datetime
    dimensions: Tuple[int, int]


m = Delivery(timestamp='2020-01-02T03:04:05Z', dimensions=['10', '20'])
print(repr(m.timestamp))
#> datetime.datetime(2020, 1, 2, 3, 4, 5, tzinfo=TzInfo(UTC))
print(m.dimensions)
#> (10, 20)

datetime.datetime(2020, 1, 2, 3, 4, 5, tzinfo=TzInfo(UTC))
(10, 20)


## Example 2

In [2]:
from datetime import datetime

from pydantic import BaseModel, PositiveInt


class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]


external_data = {
    'id': 123,
    'signup_ts': '2019-06-01 12:22',
    'tastes': {
        'wine': 9,
        b'cheese': 7,
        'cabbage': '1',
    },
}

user = User(**external_data)

print(user.id)
#> 123
print(user.model_dump())
"""
{
    'id': 123,
    'name': 'John Doe',
    'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),
    'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1},
}
"""

123
{'id': 123, 'name': 'John Doe', 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22), 'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1}}


"\n{\n    'id': 123,\n    'name': 'John Doe',\n    'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),\n    'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1},\n}\n"

## Example 3 - Validation Error

In [4]:
# continuing the above example...

from pydantic import ValidationError


class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]


external_data = {'id': 'not an int', 'tastes': {}}

try:
    User(**external_data)
except ValidationError as e:
    print(e.errors())
    """
    [
        {
            'type': 'int_parsing',
            'loc': ('id',),
            'msg': 'Input should be a valid integer, unable to parse string as an integer',
            'input': 'not an int',
            'url': 'https://errors.pydantic.dev/2/v/int_parsing',
        },
        {
            'type': 'missing',
            'loc': ('signup_ts',),
            'msg': 'Field required',
            'input': {'id': 'not an int', 'tastes': {}},
            'url': 'https://errors.pydantic.dev/2/v/missing',
        },
    ]
    """

[{'type': 'string_type', 'loc': ('id',), 'msg': 'Input should be a valid string', 'input': 2, 'url': 'https://errors.pydantic.dev/2.8/v/string_type'}, {'type': 'missing', 'loc': ('signup_ts',), 'msg': 'Field required', 'input': {'id': 2, 'tastes': {}}, 'url': 'https://errors.pydantic.dev/2.8/v/missing'}]


## Example 4

In [5]:
from typing import Annotated, Dict, List, Literal, Tuple

from annotated_types import Gt

from pydantic import BaseModel


class Fruit(BaseModel):
    name: str
    color: Literal['red', 'green']
    weight: Annotated[float, Gt(0)]
    bazam: Dict[str, List[Tuple[int, bool, float]]]


print(
    Fruit(
        name='Apple',
        color='red',
        weight=4.2,
        bazam={'foobar': [(1, True, 0.1)]},
    )
)
#> name='Apple' color='red' weight=4.2 bazam={'foobar': [(1, True, 0.1)]}

name='Apple' color='red' weight=4.2 bazam={'foobar': [(1, True, 0.1)]}


## Example 5 - Performance

In [6]:
import json
import timeit
from urllib.parse import urlparse

import requests

from pydantic import HttpUrl, TypeAdapter

reps = 7
number = 100
r = requests.get('https://api.github.com/emojis')
r.raise_for_status()
emojis_json = r.content


def emojis_pure_python(raw_data):
    data = json.loads(raw_data)
    output = {}
    for key, value in data.items():
        assert isinstance(key, str)
        url = urlparse(value)
        assert url.scheme in ('https', 'http')
        output[key] = url


emojis_pure_python_times = timeit.repeat(
    'emojis_pure_python(emojis_json)',
    globals={
        'emojis_pure_python': emojis_pure_python,
        'emojis_json': emojis_json,
    },
    repeat=reps,
    number=number,
)
print(f'pure python: {min(emojis_pure_python_times) / number * 1000:0.2f}ms')

type_adapter = TypeAdapter(dict[str, HttpUrl])
emojis_pydantic_times = timeit.repeat(
    'type_adapter.validate_json(emojis_json)',
    globals={
        'type_adapter': type_adapter,
        'HttpUrl': HttpUrl,
        'emojis_json': emojis_json,
    },
    repeat=reps,
    number=number,
)
print(f'pydantic: {min(emojis_pydantic_times) / number * 1000:0.2f}ms')

print(
    f'Pydantic {min(emojis_pure_python_times) / min(emojis_pydantic_times):0.2f}x faster'
)

pure python: 14.55ms
pydantic: 4.01ms
Pydantic 3.63x faster


## Example 6 - Serialization

In [7]:
from datetime import datetime

from pydantic import BaseModel


class Meeting(BaseModel):
    when: datetime
    where: bytes
    why: str = 'No idea'


m = Meeting(when='2020-01-01T12:00', where='home')
print(m.model_dump(exclude_unset=True)) # with 'exclude_unset=True' not getting 'why' variable because we did not give value
#> {'when': datetime.datetime(2020, 1, 1, 12, 0), 'where': b'home'}
print(m.model_dump(exclude={'where'}, mode='json'))
#> {'when': '2020-01-01T12:00:00', 'why': 'No idea'}
print(m.model_dump_json(exclude_defaults=True))# with 'exclude_defaults=True' not getting 'why' variable because it has default value
#> {"when":"2020-01-01T12:00:00","where":"home"}

{'when': datetime.datetime(2020, 1, 1, 12, 0), 'where': b'home'}
{'when': '2020-01-01T12:00:00', 'why': 'No idea'}
{"when":"2020-01-01T12:00:00","where":"home"}


## Example 7 - Json Schema

In [8]:
from datetime import datetime

from pydantic import BaseModel


class Address(BaseModel):
    street: str
    city: str
    zipcode: str


class Meeting(BaseModel):
    when: datetime
    where: Address
    why: str = 'No idea'


print(Meeting.model_json_schema())
"""
{
    '$defs': {
        'Address': {
            'properties': {
                'street': {'title': 'Street', 'type': 'string'},
                'city': {'title': 'City', 'type': 'string'},
                'zipcode': {'title': 'Zipcode', 'type': 'string'},
            },
            'required': ['street', 'city', 'zipcode'],
            'title': 'Address',
            'type': 'object',
        }
    },
    'properties': {
        'when': {'format': 'date-time', 'title': 'When', 'type': 'string'},
        'where': {'$ref': '#/$defs/Address'},
        'why': {'default': 'No idea', 'title': 'Why', 'type': 'string'},
    },
    'required': ['when', 'where'],
    'title': 'Meeting',
    'type': 'object',
}
"""

{'$defs': {'Address': {'properties': {'street': {'title': 'Street', 'type': 'string'}, 'city': {'title': 'City', 'type': 'string'}, 'zipcode': {'title': 'Zipcode', 'type': 'string'}}, 'required': ['street', 'city', 'zipcode'], 'title': 'Address', 'type': 'object'}}, 'properties': {'when': {'format': 'date-time', 'title': 'When', 'type': 'string'}, 'where': {'$ref': '#/$defs/Address'}, 'why': {'default': 'No idea', 'title': 'Why', 'type': 'string'}}, 'required': ['when', 'where'], 'title': 'Meeting', 'type': 'object'}


"\n{\n    '$defs': {\n        'Address': {\n            'properties': {\n                'street': {'title': 'Street', 'type': 'string'},\n                'city': {'title': 'City', 'type': 'string'},\n                'zipcode': {'title': 'Zipcode', 'type': 'string'},\n            },\n            'required': ['street', 'city', 'zipcode'],\n            'title': 'Address',\n            'type': 'object',\n        }\n    },\n    'properties': {\n        'when': {'format': 'date-time', 'title': 'When', 'type': 'string'},\n        'where': {'$ref': '#/$defs/Address'},\n        'why': {'default': 'No idea', 'title': 'Why', 'type': 'string'},\n    },\n    'required': ['when', 'where'],\n    'title': 'Meeting',\n    'type': 'object',\n}\n"

## Example 8 - Strict Mode and Data Coercion

In [11]:
from datetime import datetime

from pydantic import BaseModel, ValidationError


class Meeting(BaseModel):
    when: datetime
    where: bytes


m = Meeting.model_validate({'when': '2020-01-01T12:00', 'where': 'home'})
print(m)
#> when=datetime.datetime(2020, 1, 1, 12, 0) where=b'home'
try:
    m = Meeting.model_validate(
        {'when': '2020-01-01T12:00', 'where': 'home'}, strict=True
    )
except ValidationError as e:
    print(e)
    """
    2 validation errors for Meeting
    when
      Input should be a valid datetime [type=datetime_type, input_value='2020-01-01T12:00', input_type=str]
    where
      Input should be a valid bytes [type=bytes_type, input_value='home', input_type=str]
    """

m_json = Meeting.model_validate_json(
    '{"when": "2020-01-01T12:00", "where": "home"}'
)
print(m_json)
#> when=datetime.datetime(2020, 1, 1, 12, 0) where=b'home'

when=datetime.datetime(2020, 1, 1, 12, 0) where=b'home'
2 validation errors for Meeting
when
  Input should be a valid datetime [type=datetime_type, input_value='2020-01-01T12:00', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/datetime_type
where
  Input should be a valid bytes [type=bytes_type, input_value='home', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/bytes_type
when=datetime.datetime(2020, 1, 1, 12, 0) where=b'home'


## Example 9 - Dataclasses, TypedDicts, and More

In [12]:
from datetime import datetime

from typing_extensions import NotRequired, TypedDict

from pydantic import TypeAdapter


class Meeting(TypedDict):
    when: datetime
    where: bytes
    why: NotRequired[str]


meeting_adapter = TypeAdapter(Meeting)
m = meeting_adapter.validate_python(
    {'when': '2020-01-01T12:00', 'where': 'home'}
)
print(m)
#> {'when': datetime.datetime(2020, 1, 1, 12, 0), 'where': b'home'}
meeting_adapter.dump_python(m, exclude={'where'})

print(meeting_adapter.json_schema())
"""
{
    'properties': {
        'when': {'format': 'date-time', 'title': 'When', 'type': 'string'},
        'where': {'format': 'binary', 'title': 'Where', 'type': 'string'},
        'why': {'title': 'Why', 'type': 'string'},
    },
    'required': ['when', 'where'],
    'title': 'Meeting',
    'type': 'object',
}
"""

{'when': datetime.datetime(2020, 1, 1, 12, 0), 'where': b'home'}
{'properties': {'when': {'format': 'date-time', 'title': 'When', 'type': 'string'}, 'where': {'format': 'binary', 'title': 'Where', 'type': 'string'}, 'why': {'title': 'Why', 'type': 'string'}}, 'required': ['when', 'where'], 'title': 'Meeting', 'type': 'object'}


"\n{\n    'properties': {\n        'when': {'format': 'date-time', 'title': 'When', 'type': 'string'},\n        'where': {'format': 'binary', 'title': 'Where', 'type': 'string'},\n        'why': {'title': 'Why', 'type': 'string'},\n    },\n    'required': ['when', 'where'],\n    'title': 'Meeting',\n    'type': 'object',\n}\n"

## Example 10 - Customisation

In [13]:
from datetime import datetime, timezone

from pydantic import BaseModel, field_validator


class Meeting(BaseModel):
    when: datetime

    @field_validator('when', mode='wrap')
    def when_now(cls, input_value, handler):
        if input_value == 'now':
            return datetime.now()
        when = handler(input_value)
        # in this specific application we know tz naive datetimes are in UTC
        if when.tzinfo is None:
            when = when.replace(tzinfo=timezone.utc)
        return when


print(Meeting(when='2020-01-01T12:00+01:00'))
#> when=datetime.datetime(2020, 1, 1, 12, 0, tzinfo=TzInfo(+01:00))
print(Meeting(when='now'))
#> when=datetime.datetime(2032, 1, 2, 3, 4, 5, 6)
print(Meeting(when='2020-01-01T12:00'))
#> when=datetime.datetime(2020, 1, 1, 12, 0, tzinfo=datetime.timezone.utc)

when=datetime.datetime(2020, 1, 1, 12, 0, tzinfo=TzInfo(+01:00))
when=datetime.datetime(2024, 7, 8, 11, 24, 25, 141973)
when=datetime.datetime(2020, 1, 1, 12, 0, tzinfo=datetime.timezone.utc)
